# 세이지메이커 학습 작업을 사용해 대화 요약을 위한 인스트럭션 기반 학습 모델 미세 조정

# 주의: 이 노트북을 완료하는 데 약 20분이 소요됩니다.

# 잠시 기다려 주세요.

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='gaia/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# 사전 요구 사항: 이 노트북을 진행하기 전에 `준비` 섹션의 노트북을 성공적으로 실행해야 합니다.

In [3]:
%store -r processed_train_data_s3_uri

In [4]:
try:
    processed_train_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/train


In [6]:
%store -r processed_validation_data_s3_uri

In [7]:
try:
    processed_validation_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/validation


In [9]:
%store -r processed_test_data_s3_uri

In [10]:
try:
    processed_test_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [11]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/test


# S3에 있는 데이터 세트 지정하기
이전 노트북에서 생성한 학습, 검증, 테스트 분할을 사용합니다.

In [12]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/train
2023-10-25 19:35:07    2540571 1698262500203.parquet
2023-10-25 19:35:07    2545157 1698262503103.parquet


In [13]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/validation
2023-10-25 19:35:07     150701 1698262500203.parquet
2023-10-25 19:35:07     150220 1698262503103.parquet


In [14]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/test
2023-10-25 19:35:07     157115 1698262500203.parquet
2023-10-25 19:35:07     153865 1698262503103.parquet


# S3 입력 데이터 지정하기

In [15]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri)
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri)
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri)

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/train', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/validation', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-079002598131/sagemaker-scikit-learn-2023-10-25-19-28-08-587/output/test', 'S3DataDistributionType': 'FullyReplicated'}}}


# FLAN 모델을 위한 하이퍼파라미터 설정하기

In [16]:
model_checkpoint='google/flan-t5-base'

In [17]:
epochs = 1 # 더 오랜 기간 동안 학습하고 싶다면 이 값을 늘리세요.
learning_rate = 0.00001
weight_decay = 0.01
train_batch_size = 4
validation_batch_size = 4
test_batch_size = 4
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
input_mode = "FastFile"
train_sample_percentage = 0.01 # 더 많은 데이터로 학습하고 싶다면 이 값을 늘리세요.

# 모델 성능 추적을 위한 메트릭 설정하기

In [18]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "'train_loss': ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "'eval_loss': ([0-9\\.]+)"},
]

# 체크포인트 S3 위치 지정하기
이 노트북에는 스팟 인스턴스 학습을 사용합니다. 노드가 교체되면, 새 노드는 최신 체크포인트에서 학습을 시작합니다.

In [19]:
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-us-west-2-079002598131/checkpoints/13aab899-f629-4e25-a856-0fbfe8dce688/


# 세이지메이커에서 실행할 스크립트 설정하기
관리형 세이지메이커 서비스에서 모델을 실행할 준비합니다.

In [20]:
!pygmentize src/train.py

import argparse
import os
import json
import pprint

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
from datasets import load_dataset

def list_files(startpath):
    """Helper function to list files in a directory"""
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--validation_data", type=str, default=os.environ["SM_CHANNEL_VALIDATION"])
    parser.add_argument("--test_data", type=str, default=os.environ["SM_CHANNEL_TEST"])
    parser.add_argument("--output_dir", type=str, default=os.environ[

In [21]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="train.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py39",
    framework_version="1.13",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,        
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "model_checkpoint": model_checkpoint,
        "train_sample_percentage": train_sample_percentage,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# 세이지메이커에서 모델 학습하기

In [22]:
estimator.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    wait=False,
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-10-25-19-41-48-426


Using provided s3_resource


In [23]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name: {}".format(training_job_name))

Training Job Name: pytorch-training-2023-10-25-19-41-48-426


In [24]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

/tmp/ipykernel_518/197746413.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [25]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

/tmp/ipykernel_518/4162796959.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [26]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

/tmp/ipykernel_518/728904872.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [27]:
%%time

estimator.latest_training_job.wait(logs=False)


2023-10-25 19:41:49 Starting - Starting the training job.
2023-10-25 19:42:03 Starting - Preparing the instances for training..............
2023-10-25 19:43:18 Downloading - Downloading input data....
2023-10-25 19:43:43 Training - Downloading the training image..........................
2023-10-25 19:45:59 Training - Training image download completed. Training in progress...........................................
2023-10-25 19:49:35 Uploading - Uploading generated training model..
2023-10-25 19:49:51 Completed - Training job completed
CPU times: user 568 ms, sys: 69.2 ms, total: 637 ms
Wall time: 8min 4s


# 미세 조정된 모델을 실시간 엔드포인트에 배포하기

In [28]:
sm_model = estimator.create_model(
    entry_point='inference.py',
    source_dir='src',
)
endpoint_name = training_job_name.replace('pytorch-training', 'summary-tuned')
predictor = sm_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.2xlarge',
    endpoint_name=endpoint_name
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-west-2-079002598131/pytorch-training-2023-10-25-19-41-48-426/output/model.tar.gz), script artifact (src), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-west-2-079002598131/pytorch-training-2023-10-25-19-49-54-208/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-10-25-19-49-54-208
INFO:sagemaker:Creating endpoint-config with name summary-tuned-2023-10-25-19-41-48-426
INFO:sagemaker:Creating endpoint with name summary-tuned-2023-10-25-19-41-48-426


-----!

# 세이지메이커 엔드포인트에서 미세 조정된 모델로 제로샷 추론하기

In [29]:
zero_shot_prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""

In [30]:
import json
from sagemaker import Predictor
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
)
response = predictor.predict(zero_shot_prompt,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
)
response_json = json.loads(response.decode('utf-8'))
print(response_json)

Tom's novel has won the Nobel Prize.


# 엔드포인트 종료하기

In [31]:
# predictor.delete_endpoint()